# DataSet:

In [1]:
Train_data = [['Green',3,'Mango'],['Yellow',3,'Mango'],['Red',1,'Grape'],['Red',1,'Grape'],['Yellow',3,'Lemon'] ] 

In [2]:
# Columns labls
Headers = ["Color","Diameter","Label"]

In [3]:
# unique_values: find the unique values for a column in data

def uniques_values(rows,col):
    return set([row[col] for row in rows])

In [4]:
uniques_values(Train_data,0)

{'Green', 'Red', 'Yellow'}

In [5]:
uniques_values(Train_data,1)

{1, 3}

In [9]:
# Class_count: count the number of each type in the data.

def class_count(rows):
    
    count={}
    for row in rows:
        label=row[-1]
        if label not in count:
            count[label]=0
            
        count[label]+=1
    
    
    return count

In [10]:
class_count(Train_data)

{'Mango': 2, 'Grape': 2, 'Lemon': 1}

In [11]:
# is_numeric: test if a value is numeric.

def is_numeric(val):
    return isinstance(val,int) or isinstance(val, float)

In [12]:
is_numeric(7)

True

In [15]:
is_numeric('7')

False

In [ ]:
class Question:
    """ A question is used to partition a dataset.
    
        this class record a column number and column value. 
        the match method is used to compare a feature value in an exemple to the feature 
        value stored in the question."""
    
    def __init__(self, column, value):
        self.column = column
        self.value = value
        
    def match(self, example):
        # compare the example value with the value in the question.
        
        val = example[self.column]
        
        